# Satellite Imagery from Sentinel 2

Before running the notebook, do the following steps first (following instructions and code [here](https://github.com/artefactory/medium_satellite_imagery)):

*   create a geojson file of the zone you want a satellite imagery of and save it in 1_collection/city.geojson
*   create an account at Coppernicus Open Access Hub. Write the username and the password in a json file and save it in 1_collection/sentinel_api_credentials.json. Format: {"username": "XXXX", "password": "XXXX"}




## Step 0: Requirements & Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install rasterio
!pip install sentinelsat

In [ ]:
import os 
import cv2
from datetime import date
import json
import itertools
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import pandas as pd

## Step 1: Authentication with Copernicus Open Access Hub

In [ ]:
with open("./sentinel_api_credentials.json", "r") as infile:
    credentials = json.load(infile)

api = SentinelAPI(credentials["username"], credentials["password"], "https://scihub.copernicus.eu/dhus")

## Step 2: Define area of interest in a geojson file

A way to specify the zones you want to explore is to provide the API with a geojson file, which is a json containing GPS coordinates position (latitude and longitude) of a zone. Geojson can be quickly created on this [website](https://geojson.io/#map=2/20/0).

In [ ]:
geojson_path = "./kyiv.geojson"

## Step 3: Query satellite images via Sentinelsat API

In [ ]:
shape = geojson_to_wkt(read_geojson(geojson_path))

images = api.query(
    shape,
    date=(date(2021, 9, 1), date(2022, 1, 30)),
    platformname="Sentinel-2",
    processinglevel = "Level-2A",
    cloudcoverpercentage=(0, 5)
)

images_df = api.to_dataframe(images)

We can use the is_online() function from the sentinelsat package to filter the queried images that are available online. A for loop iterates over the images dictionary and checks if each image is online. 

In [ ]:
online_images = {}
for id, item in images.items():
    if api.is_online(id):
        online_images[id] = item

# Convert the filtered images to a dataframe
online_images_df = api.to_dataframe(online_images)

In [ ]:
key_columns = ["platformserialidentifier", "processinglevel", "cloudcoverpercentage", 
               "vegetationpercentage", "beginposition"]
online_images_df = online_images_df.sort_values(by=['cloudcoverpercentage'], ascending=True)
online_images_df[key_columns].head(5)

,platformserialidentifier,processinglevel,cloudcoverpercentage,vegetationpercentage,beginposition
0194bc53-5048-46c5-8d9e-135fcca9a15d,Sentinel-2A,Level-2A,0.030464,19.858183,2021-11-01 09:11:31.024
efb95b8c-acaf-4012-ae90-2e7dd4a336ae,Sentinel-2A,Level-2A,0.123288,66.410869,2021-09-09 08:56:01.024
09b4ef40-2eb0-4fcf-bf31-0fb1588def6c,Sentinel-2A,Level-2A,0.334950,5.633983,2021-11-01 09:11:31.024
1c4cdac1-6a69-46ad-b088-56a1a6888a47,Sentinel-2A,Level-2A,0.460456,4.522084,2021-11-01 09:11:31.024
b6fd658c-4e7e-4bff-a7e2-d71d1850f04a,Sentinel-2B,Level-2A,1.362930,8.289550,2022-01-02 09:02:59.024


##  Step 4: Download satellite image

In [ ]:
uuid = "efb95b8c-acaf-4012-ae90-2e7dd4a336ae"
api.download(uuid)

## Step 5: Unzip satellite image and save to folder

In [ ]:
!unzip /content/S2A_MSIL2A_20210909T085601_N0301_R007_T35UQR_20210909T120232.zip -d ../data/sentinel2/kyiv

# Credits

*   Antoine Aubay (2021): Leveraging satellite imagery for machine learning computer vision applications. Available at: https://medium.com/artefact-engineering-and-data-science/leveraging-satellite-imagery-for-machine-learning-computer-vision-applications-d22143f72d94

